# Notebook for Climate Data 

In [ ]:
import xarray as xr
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 

#Load Climate Modell 
data_pr = xr.open_dataset('../data/climate_model/PR_data_climatemodel.nc')
data_wind = xr.open_dataset('../data/climate_model/sfcWind_EUR-11_MPI-M-MPI-ESM-LR_rcp26_r2i1p1_DWD-EPISODES2018_v1-r1_mon_200601-210012.nc')
wind = data_wind.sfcWind
pr = data_pr.pr

In [27]:
# Datenpunkte 
data_pr.dims
# Datenvariablen
data_pr.data_vars
#ausfürhliche Version im Text Editor anschauen sonst unvollständig
pr.shape
#data_pr.info()




(1140, 412, 424)

In [ ]:
# Datenpunkte 
data_wind.dims
# Datenvariablen
data_wind.data_vars
#data_wind.info()
#wind.shape

In [ ]:
#Metadaten Klima-Modell etc.
#da ähnliche Parameter eig identisch
data_pr.attrs
data_wind.attrs

In [ ]:
#Useful Commands 
#ds.air.mean(dim="time").plot(x="lon")
## pull out time index 0, lat index 2, and lon index 3
# ds.air.isel(time=0, lat=2, lon=3)  #  much better than ds.air[0, 2, 3]
# Indexierter Zugriff
#First Precipitation Data whole Germany 
# 500 Zeiteintrag (in Datum) und zehnter rat (in Grad)
# Zugriff auf Stelle in den Daten nicht auf Werte
#data_pr.pr[500].plot()


In [ ]:
#sel select values by label 
#Slice Right Time 
data_pr_2030 = data_pr.sel(time=slice("2024-01-16","2054-01-16"))
# Slice degrees beiing out of germany
data_pr_2030_zoom = data_pr_2030.sel(rlon=slice(-9,-1), rlat=slice(-5,6))
data_pr_2030_zoom.pr.coords
data_pr_2030_zoom.pr[1].plot()
#data_pr_2030_zoom.pr[5].plot()
#data_pr_2030_zoom.pr[200].plot()
#data_pr_2030_zoom.pr

In [ ]:
wind = wind.sel(time=slice("2024-01-16","2054-01-16"), rlon=slice(-9,-1), rlat=slice(-5,6))
wind.shape
wind[1,:,:].plot()